In [14]:
import numpy as np
import pandas as pd

In [16]:
movies=pd.read_csv('tmdb_5000_movies.csv')
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [18]:
from sklearn.preprocessing import LabelEncoder
movies.iloc[:,1]=LabelEncoder().fit_transform(movies.iloc[:,1])
new_movie=movies.groupby('genres').agg('count')['title'].reset_index().rename({'title':'title count'},axis=1)
new_movie

,genres,title count
0,0,28
1,1,3
2,2,1
3,3,1
4,4,4
5,5,1
6,6,2
7,7,1
8,8,5
9,9,1


In [22]:
final_movies=movies.merge(new_movie,on='genres')[['genres','title','vote_average','vote_count','title count']]
final_movies=final_movies[final_movies['vote_average']>=5.5]
#from sklearn.preprocessing import OneHotEncoder
#final_movies=OneHotEncoder(categorical_features=[0]).fit_transform(final_movies).toarray()
final_pivot=final_movies.pivot_table(values='vote_average',index='title',columns='genres').fillna(0)

In [23]:
from scipy.sparse import csr_matrix
movies_sparse=csr_matrix(final_pivot)

In [29]:
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(metric='cosine',algorithm='brute')
nn.fit(movies_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [37]:
np.where(final_pivot.index=='The Godfather')[0][0]

2943

In [40]:
distance,suggestion=nn.kneighbors(final_pivot.iloc[2943,:].values.reshape(1,-1),n_neighbors=7)

In [41]:
suggestion=final_pivot.index[suggestion].ravel()

In [42]:
for i in range(len(suggestion)):
    print([suggestion[i]])

['Wall Street: Money Never Sleeps']
['American Gangster']
['21']
['Trainspotting']
['The Place Beyond the Pines']
['GoodFellas']
["Things to Do in Denver When You're Dead"]
